## Solving Systems using Elimination


In this section we demonstrate all of the code needed to solve the linear system $AX=B$ using elimination, in the case that $A$ is a square $n\times n$ matrix.  We will consder the case where $A$ is $m\times n$ at a later point.

In [1]:
import numpy as np
import laguide as lag

### Row Reduction routine

Elimination is currently our only method of solution. It is also a common task that will arise in future sections.  Let's build a Python function that will carry out all the steps of elimination, and just return the end result.  If we want the code to be able to handle any array, we will need to include instructions to _make the decision_ of whether or not to perform row swaps.  We would also like to be able to carry out the elimination on arrays of any size or shape.  

In [7]:
def RowReduction(A):
    ''' 
    RowReduction performs steps of elimination with no pivot strategy and
    returns a row echelon form of the matrix A.
    
    Parameters
    ----------
    A : NumPy array object of dimension mxn
    
    Returns
    -------
    B: NumPy array object of dimension mxn
    '''
    
    m = A.shape[0]  # m is number of rows in A
    n = A.shape[1]  # n is number of columns in A

    B = np.zeros((m,n))
    for i in range(m):
        for j in range(n):
            B[i][j] = A[i][j]

    if m < n:
        elimination_steps = m
    else:
        elimination_steps = n

    # For each step of elimination, we find a suitable pivot, move it into
    # position and create zeros for all entries below.
    
    for k in range(elimination_steps):
        # Set pivot as (k,k) entry
        pivot = B[k][k]
        pivot_row = k
        
        # Find a suitable pivot if the (k,k) entry is zero
        while(pivot == 0 and pivot_row < m):
            pivot_row += 1
            pivot = B[pivot_row][k]
            
        # Swap row if needed
        if (pivot_row != k):
            B = RowSwap(B,k,pivot_row)
            
        # If pivot is nonzero, carry on with elimination in column k
        if (pivot != 0):
            B = lag.RowScale(B,k,1./B[k][k])
            for i in range(k+1,m):    
                B = lag.RowAdd(B,k,i,-B[i][k])
    return B
    


Let's test the routine on a random array.

In [8]:
R = np.random.randint(-8,8,size=(3,3))
print(R)
print('\n')
print(RowReduction(R))

[[ 6 -6 -7]
 [ 2  0  6]
 [-6  1 -2]]


[[ 1.         -1.         -1.16666667]
 [ 0.          1.          4.16666667]
 [ 0.          0.          1.        ]]


### System Solve routine

Now we can combine the $\texttt{RowReduction}$ and the $\texttt{BackSubstitution}$ routines together to carry out the solution algorithm for the system $AX=B$.  Since we will be the users of this function, let's try to make it easy to use.  One simple possibility is that the user of the function will supply $A$ and $B$, and the function will return the solution $X$.  Let's list the steps that need to be completed.

1. Build an augmented matrix.
2. Apply $\texttt{RowReduction}$.
3. Split the matrix.
4. Apply $\texttt{BackSubstitution}$ and return the result. 

In [9]:
def SolveSystem(A,B):
    ''' 
    SystemSolve computes the solution to AX=B by elimination in the case that
    A is a square nxn matrix
    
    Parameters
    ----------
    A : NumPy array object of dimension nxn
    B : NumPy array object of dimension nx1
    
    Returns
    -------
    X: NumPy array object of dimension nx1
    '''
    # Check shape of A
    if (A.shape[0] != A.shape[1]):
        print("SolveSystem accepts only square arrays.")
        return
    n = A.shape[0]  # n is number of rows and columns in A

    # Join A and B to make the augmented matrix
    A_augmented = np.hstack((A,B))
    
    # Carry out elimination    
    R = RowReduction(A_augmented)

    # Split R back to nxn piece and nx1 piece
    B_reduced = R[:,n:n+1]
    A_reduced = R[:,0:n]

    # Do back substitution
    X = lag.BackSubstitution(A_reduced,B_reduced)
    return X

Let's test the routine by building a random matrix, choosing a solution, and constructing a system $AX=B$.

In [10]:
A = np.array([[1,2,3],[0,1,-2],[3,3,-2]])
X_true = np.array([[1],[1],[1]])
B = A@X_true
X = SolveSystem(A,B)
print(X)

[[1.]
 [1.]
 [1.]]


### Exercise, or further example?

Modify $\texttt{RowReduction}$ to compute rref and use the modified routine to solve the system.
